# Libraries

In [78]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

In [79]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Import data

In [80]:
%run Data_preprocessing.ipynb

0
0
0


In [104]:
X_train

LEVEL2               alanine aminotransferase                                \
Aggregation Function                     mask                                 
hours_in                                    0    1    2    3    4    5    6   
3_145834_211552                           1.0  0.0  0.0  0.0  0.0  0.0  0.0   
6_107064_228232                           1.0  0.0  0.0  0.0  0.0  0.0  0.0   
9_150750_220597                           0.0  0.0  0.0  0.0  0.0  0.0  0.0   
11_194540_229441                          0.0  0.0  0.0  0.0  0.0  0.0  0.0   
12_112213_232669                          0.0  0.0  0.0  0.0  0.0  0.0  0.0   
...                                       ...  ...  ...  ...  ...  ...  ...   
99966_167228_252173                       0.0  0.0  0.0  0.0  0.0  0.0  0.0   
99973_150202_275083                       0.0  0.0  0.0  0.0  0.0  0.0  0.0   
99982_151454_221194                       0.0  0.0  0.0  0.0  0.0  0.0  0.0   
99991_151118_226241                       0.0  0.0  0.0  0.0  0.0  0.0  0.0   
99995_137810_229633                       0.0  1.0  1.0  0.0  0.0  0.0  0.0   

LEVEL2                               ... white blood cell count urine         \
Aggregation Function                 ...          time_since_measured          
hours_in                7    8    9  ...                           14     15   
3_145834_211552       0.0  0.0  0.0  ...                          6.0    7.0   
6_107064_228232       0.0  0.0  0.0  ...                         30.0   31.0   
9_150750_220597       0.0  0.0  0.0  ...                         54.0   55.0   
11_194540_229441      0.0  0.0  0.0  ...                         78.0   79.0   
12_112213_232669      0.0  0.0  0.0  ...                        102.0  103.0   
...                   ...  ...  ...  ...                          ...    ...   
99966_167228_252173   0.0  0.0  0.0  ...                         86.0   87.0   
99973_150202_275083   0.0  0.0  0.0  ...                        110.0  111.0   
99982_151454_221194   0.0  0.0  0.0  ...                        134.0  135.0   
99991_151118_226241   0.0  0.0  0.0  ...                         14.0   15.0   
99995_137810_229633   0.0  0.0  0.0  ...                         38.0   39.0   

LEVEL2                                                                        
Aggregation Function                                                          
hours_in                 16     17     18     19     20     21     22     23  
3_145834_211552         8.0    9.0   10.0   11.0   12.0   13.0   14.0   15.0  
6_107064_228232        32.0   33.0   34.0   35.0   36.0   37.0   38.0   39.0  
9_150750_220597        56.0   57.0   58.0   59.0   60.0   61.0   62.0   63.0  
11_194540_229441       80.0   81.0   82.0   83.0   84.0   85.0   86.0   87.0  
12_112213_232669      104.0  105.0  106.0  107.0  108.0  109.0  110.0  111.0  
...                     ...    ...    ...    ...    ...    ...    ...    ...  
99966_167228_252173    88.0   89.0   90.0   91.0   92.0   93.0   94.0   95.0  
99973_150202_275083   112.0  113.0  114.0  115.0  116.0  117.0  118.0  119.0  
99982_151454_221194   136.0  137.0  138.0  139.0  140.0  141.0  142.0  143.0  
99991_151118_226241    16.0   17.0   18.0   19.0   20.0   21.0   22.0   23.0  
99995_137810_229633    40.0   41.0   42.0   43.0   44.0   45.0   46.0   47.0  

[16760 rows x 7488 columns]

# Hyperparameters

In [82]:
input_size = 312 # 7488/24
sequence_length = 24
output_size = 2 # live or dead
num_layers = 2
hidden_size=256
learning_rate = 0.01
batch_size = 64
num_epochs = 2

In [83]:
7488/24

312.0

# Create RNN

In [84]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size*sequence_length, num_epochs)
        # N * time_seq * features
        
        
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0),self.hidden_size).to(device)
        out, _ = self.rnn(x,h0)
        out = out.reshape(out.shape[0],-1)
        out = (F.softmax(self.fc(out), dim=1))
        #out = torch.argmax(out, dim=1)
        #print(out)
        return out

# Load data

In [85]:
class CusDatasetLoader(Dataset):
    def __init__(self,x,y):
        self.len = y.shape[0]
        self.x_data = x
        self.y_data = y
  
    def __len__(self):
        return self.len

    def __getitem__(self, index):
        #return self.dataframe.iloc[index]
        return self.x_data[index], self.y_data[index]

In [86]:
def X_split(X):
    X_tensor = torch.from_numpy(np.array(X)).to(torch.float32)
    print(X_tensor.shape)
    X_tensor = torch.split(X_tensor,24,1)
    X_tensor = torch.stack(X_tensor)
    print(X_tensor.shape)
    return X_tensor

def y_tensor(y):
    y= torch.from_numpy(np.array(y)).to(torch.float32)
    return y

In [88]:
# X should be in smaples, 1, squence, rows
X1 = X_split(X_train)
X1 = X1.permute(1, 0, 2)
print(X1.shape)
y1 = y_tensor(y_train_t1.iloc[:,-1]).reshape(len(y_train_t1.iloc[:,-1]),1)
print(y1.shape)
train_datasets = CusDatasetLoader(X1, y1)
train_loader = DataLoader(dataset=train_datasets, batch_size=batch_size, shuffle=True)

torch.Size([16760, 7488])
torch.Size([312, 16760, 24])
torch.Size([16760, 312, 24])
torch.Size([16760, 1])


In [89]:
x,y = train_datasets[0]
print(x.shape)
print(y.shape)

torch.Size([312, 24])
torch.Size([1])


In [90]:
model = RNN(input_size, hidden_size, num_layers, output_size).to(device)

In [91]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

# Model train

In [97]:
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(train_loader):
        data = data.permute(0,2,1)
        #print(data.shape)
        data = data.to(device=device)
        targets = targets.to(device=device)
        targets = torch.squeeze(targets, 1).type(torch.LongTensor)
        #print(targets)
        
        # forward
        scores = model(data)
        #print(scores.shape)
        loss = criterion(scores, targets)
        
        # backward
        optimizer.zero_grad()
        loss.backward()
        
        # gradient descent or adam step
        optimizer.step()
        
        

# Model Accuracy

In [129]:
# Check accuracy on training & test to see how good our model
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0

    # Set model to eval
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device).permute(0,2,1)
            #print(x.shape)
            y = torch.squeeze(y.to(device=device), 1)
            #print(y)

            scores = model(x)
            _, predictions = scores.max(1)
            #print(predictions)
            num_correct += int((predictions == y).sum())
            #print(num_correct)
            num_samples += predictions.size(0)
            
    # Toggle model back to train
    model.train()
    #print(num_correct , num_samples)
    return num_correct / num_samples


print(f"Accuracy on training set: {check_accuracy(train_loader, model)*100:2f} %")
#print(f"Accuracy on test set: {check_accuracy(test_loader, model)*100:.2f} %")

Accuracy on training set: 92.690931 %


In [130]:
X2 = X_split(X_valid)
X2 = X2.permute(1, 0, 2)
print(X2.shape)
y2 = y_tensor(y_valid_t1.iloc[:,-1]).reshape(len(y_valid_t1.iloc[:,-1]),1)
print(y2.shape)
Valid_datasets = CusDatasetLoader(X2, y2)
Valid_loader = DataLoader(dataset=train_datasets, batch_size=batch_size, shuffle=True)
print(f"Accuracy on test set: {check_accuracy(Valid_loader, model)*100:.2f} %")

torch.Size([2394, 7488])
torch.Size([312, 2394, 24])
torch.Size([2394, 312, 24])
torch.Size([2394, 1])
Accuracy on test set: 92.69 %


#

In [93]:
a = torch.tensor([[0.6316, 0.3684],
        [0.5012, 0.4988],
        [0.6390, 0.3610],
        [0.6141, 0.3859],
        [0.6435, 0.3565],
        [0.4579, 0.5421],
        [0.6019, 0.3981],
        [0.5691, 0.4309],
        [0.6579, 0.3421],
        [0.5140, 0.4860],
        [0.5985, 0.4015],
        [0.4886, 0.5114],
        [0.5201, 0.4799],
        [0.4950, 0.5050],
        [0.6365, 0.3635],
        [0.5573, 0.4427],
        [0.6125, 0.3875],
        [0.4695, 0.5305],
        [0.5096, 0.4904],
        [0.6424, 0.3576],
        [0.5129, 0.4871],
        [0.5582, 0.4418],
        [0.5545, 0.4455],
        [0.5052, 0.4948],
        [0.5731, 0.4269],
        [0.6821, 0.3179],
        [0.5329, 0.4671],
        [0.5993, 0.4007],
        [0.5270, 0.4730],
        [0.6330, 0.3670],
        [0.5310, 0.4690],
        [0.6179, 0.3821],
        [0.6537, 0.3463],
        [0.6906, 0.3094],
        [0.5360, 0.4640],
        [0.5581, 0.4419],
        [0.5531, 0.4469],
        [0.6437, 0.3563],
        [0.5725, 0.4275],
        [0.6203, 0.3797],
        [0.6588, 0.3412],
        [0.3213, 0.6787],
        [0.5850, 0.4150],
        [0.5092, 0.4908],
        [0.6125, 0.3875],
        [0.5573, 0.4427],
        [0.6332, 0.3668],
        [0.5935, 0.4065],
        [0.5413, 0.4587],
        [0.5539, 0.4461],
        [0.5196, 0.4804],
        [0.4552, 0.5448],
        [0.6290, 0.3710],
        [0.4886, 0.5114],
        [0.5645, 0.4355],
        [0.6414, 0.3586],
        [0.6594, 0.3406],
        [0.5228, 0.4772],
        [0.5270, 0.4730],
        [0.5980, 0.4020],
        [0.6281, 0.3719],
        [0.4989, 0.5011],
        [0.5402, 0.4598],
        [0.6284, 0.3716]])
print(a.shape)
torch.argmax(a , dim=1)

torch.Size([64, 2])


tensor([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0])

#

#

#

#

#